## customers table

In [0]:
query = "SELECT * FROM databricks_simulated_retail_customer_data.v01.customers"
customers_df = spark.sql(query)
display(customers_df)

Databricks data profile. Run in Databricks to view.

In [0]:
# 必要ないカラムの削除
exclude_cols = [
    "tax_id", "tax_code", "customer_name", "postcode", "street", "number",
    "unit", "region", "district", "lon", "lat", "ship_to_address", "valid_from", "valid_to"
]

selected_cols = [col for col in customers_df.columns if col not in exclude_cols]

print("[" + ", ".join(selected_cols) + "]")

In [0]:
%sql
SELECT
  customer_id,
  state,
  city,
  units_purchased,
  loyalty_segment
FROM
  databricks_simulated_retail_customer_data.v01.customers
where
  units_purchased is not null
  and loyalty_segment is not null
  and state is not null
  and city is not null
  and trim(units_purchased) != ''
  and trim(loyalty_segment) != ''
  and trim(state) != ''
  and trim(city) != ''

In [0]:
query = """
SELECT
  customer_id,
  state,
  city,
  units_purchased,
  loyalty_segment
FROM
  databricks_simulated_retail_customer_data.v01.customers
where
  units_purchased is not null
  and loyalty_segment is not null
  and state is not null
  and city is not null
  and trim(units_purchased) != ''
  and trim(loyalty_segment) != ''
  and trim(state) != ''
  and trim(city) != ''
"""

customers_df_cleansing = spark.sql(query)
display(customers_df_cleansing)

## sales table

In [0]:
query = "SELECT * FROM databricks_simulated_retail_customer_data.v01.sales"
sales_df = spark.sql(query)
display(sales_df)

Databricks data profile. Run in Databricks to view.

In [0]:
# 必要ないカラムの削除
exclude_cols = [
    "customer_name"
]

selected_cols = [col for col in sales_df.columns if col not in exclude_cols]

print("[" + ", ".join(selected_cols) + "]")

In [0]:
query = """
SELECT
  customer_id,
  product_name,
  order_date,
  product_category,
  product,
  total_price
FROM
  databricks_simulated_retail_customer_data.v01.sales
WHERE
  customer_id IS NOT NULL
  AND product_name IS NOT NULL
  AND order_date IS NOT NULL
  AND product_category IS NOT NULL
  AND product IS NOT NULL
  AND total_price IS NOT NULL
  AND TRIM(customer_id) != ''
  AND TRIM(product_name) != ''
  AND TRIM(order_date) != ''
  AND TRIM(product_category) != ''
  AND TRIM(product) != ''
  AND TRIM(total_price) != ''
"""

sales_df_cleansing = spark.sql(query)
display(sales_df_cleansing)

## sales_orders table

In [0]:
query = "SELECT * FROM databricks_simulated_retail_customer_data.v01.sales_orders"
sales_orders_df = spark.sql(query)
display(sales_orders_df)

Databricks data profile. Run in Databricks to view.

In [0]:
# 必要ないカラムの削除
exclude_cols = [
    "customer_name"
]

selected_cols = [col for col in sales_orders_df.columns if col not in exclude_cols]

print("[" + ", ".join(selected_cols) + "]")

In [0]:
query = """
SELECT
  clicked_items,
  customer_id,
  number_of_line_items,
  from_unixtime(CAST(order_datetime AS BIGINT)) AS order_datetime,
  order_number,
  ordered_products,
  promo_info
FROM
  databricks_simulated_retail_customer_data.v01.sales_orders
WHERE
  customer_id IS NOT NULL
  AND clicked_items IS NOT NULL
  AND number_of_line_items IS NOT NULL
  AND order_datetime IS NOT NULL
  AND order_number IS NOT NULL
  AND ordered_products IS NOT NULL
  AND promo_info IS NOT NULL
  AND TRIM(customer_id) != ''
  AND TRIM(clicked_items) != ''
  AND TRIM(number_of_line_items) != ''
  AND TRIM(order_datetime) != ''
  AND TRIM(order_number) != ''
  AND TRIM(ordered_products) != ''
  AND TRIM(promo_info) != ''
"""

sales_orders_df_cleansing = spark.sql(query)
display(sales_orders_df_cleansing)

In [0]:
# query = """
# WITH base AS (
#   SELECT
#     clicked_items,
#     customer_id,
#     number_of_line_items,
#     from_unixtime(CAST(order_datetime AS BIGINT)) AS order_datetime,
#     order_number,
#     ordered_products,
#     promo_info
#   FROM
#     databricks_simulated_retail_customer_data.v01.sales_orders
#   WHERE
#     customer_id IS NOT NULL
#     AND clicked_items IS NOT NULL
#     AND number_of_line_items IS NOT NULL
#     AND order_datetime IS NOT NULL
#     AND order_number IS NOT NULL
#     AND ordered_products IS NOT NULL
#     AND promo_info IS NOT NULL
#     AND TRIM(customer_id) != ''
#     AND TRIM(clicked_items) != ''
#     AND TRIM(number_of_line_items) != ''
#     AND TRIM(order_datetime) != ''
#     AND TRIM(order_number) != ''
#     AND TRIM(ordered_products) != ''
#     AND TRIM(promo_info) != ''
# ),
# exploded AS (
#   SELECT
#     clicked_items,
#     customer_id,
#     number_of_line_items,
#     order_datetime,
#     order_number,
#     promo_info,
#     p.curr  AS currency,
#     p.id    AS product_id,
#     p.name  AS product_name,
#     p.price AS price,
#     p.promotion_info AS item_promotion_info,
#     p.qty   AS qty,
#     p.unit  AS unit
#   FROM base
#   LATERAL VIEW EXPLODE(FROM_JSON(ordered_products,
#     'ARRAY<STRUCT<curr:STRING,id:STRING,name:STRING,price:STRING,promotion_info:STRING,qty:STRING,unit:STRING>>')) t AS p
# )
# SELECT * FROM exploded
# """

# sales_orders_df_cleansing = spark.sql(query)
# display(sales_orders_df_cleansing)


## table join

In [0]:
joined_df = customers_df_cleansing.join(
    sales_df_cleansing, on="customer_id", how="inner"
).join(
    sales_orders_df_cleansing, on="customer_id", how="inner"
)

display(joined_df)